In [1]:
import pandas as pd
from pathlib import Path

In [2]:
file = 'full_scrape_20yr'
meta_df = pd.read_csv(file)



In [3]:
meta_df

,artist,album,date,week_num,meta_score,user_score,pickle_name,album_img,crit_rev_num,user_rev_num,record_label,album_genre
0,The Streets,Original Pirate Material,"October 22, 2002",43,90,87,Original Pirate Material-The Streets,https://static.metacritic.com/images/products/...,25.0,87,Locked On / Vice,Electronic
1,Tom Waits,Alice,"May 7, 2002",19,90,85,Alice-Tom Waits,https://static.metacritic.com/images/products/...,20.0,86,Epitaph / Anti-,Rock
2,Queens of the Stone Age,Songs For The Deaf,"August 27, 2002",35,89,89,Songs For The Deaf-Queens of the Stone Age,https://static.metacritic.com/images/products/...,23.0,89,Interscope,Rock
3,Spoon,Kill The Moonlight,"August 20, 2002",34,88,88,Kill The Moonlight-Spoon,https://static.metacritic.com/images/products/...,21.0,88,Merge,Indie
4,Wilco,Yankee Hotel Foxtrot,"April 23, 2002",17,87,89,Yankee Hotel Foxtrot-Wilco,https://static.metacritic.com/images/products/...,26.0,89,Nonesuch,Rock
...,...,...,...,...,...,...,...,...,...,...,...,...
9314,Bonny Doon,Longwave,"March 23, 2018",12,68,0,Longwave-Bonny Doon,https://static.metacritic.com/images/products/...,8.0,0,Woodsist,Pop/Rock
9315,A Perfect Circle,Eat the Elephant,"April 20, 2018",16,68,73,Eat the Elephant-A Perfect Circle,https://static.metacritic.com/images/products/...,15.0,73,BMG,Alternative/Indie Rock
9316,Arthur Buck,Arthur Buck,"June 15, 2018",24,68,0,Arthur Buck-Arthur Buck,https://static.metacritic.com/images/products/...,10.0,0,New West,Pop/Rock
9317,Josh Rouse,Love in the Modern Age,"April 13, 2018",15,68,0,Love in the Modern Age-Josh Rouse,https://static.metacritic.com/images/products/...,8.0,0,Yep Roc,Pop/Rock
